##Using HackaMol to calculate the Ramachandran Plot

This code runs well with 1AXC. There is fair resemblance between the RStudio plot and http://www.rcsb.org/pdb/images/1AXC_ram_m_500.pdf .
While for other pdbs the result is not duplicated as effectively.
Next course of action will be designing a phi,psi funtion that gives only one phi,psi plot for each residue, as opposed to multiple plots for the same residue. 


In [1]:
use Modern::Perl;
use HackaMol;

### Load a molecule within multiple chains from the pdbid

In [6]:
my $pdbid    = '4LGL';
my $mol      = HackaMol->new->pdbid_mol("$pdbid");
my @bb_atoms = grep {$_->occ == 1.0} $mol->select_atoms(sub{
                                        $_ -> name eq 'C' or
                                        $_ -> name eq 'CA' or
                                        $_ -> name eq 'N' 
                                      });
                                     

Warning: 4LGL.pdb exists, set self->overwrite(1) to overwrite at /Users/chem_student/perl5/perlbrew/perls/perl-5.20.1/lib/site_perl/5.20.1/HackaMol.pm line 26.


## Create groups of atoms according to their chains

In [8]:
my @chains = HackaMol->new->group_by_atom_attr('chain',@bb_atoms);
my $bb_mol = HackaMol::Molecule->new(groups => [@chains]);
$bb_mol->print_pdb ("$pdbid\_backbone.pdb");1

1


Warning: overwrite 4LGL_backbone.pdb at reply input line 3.


##Construct the backbone dihedrals for each chain
The backbone atoms are ordered in repeats {N, CA, C}. Walking down the backbone: N0-CA0-C0-N1-CA1-C1-N2-CA2-C2 ...there will be dihedrals about the CA0-C0(PSI), C0-N1(OMEGA), N1-CA1 (PHI), CA1-C1 (PSI), ... bonds.  Since there are no bonds between chains, they must be treated separately.

In [9]:
my @dihedrals;
foreach my $chain ($bb_mol->all_groups){
    push @dihedrals, HackaMol->new->build_dihedrals($chain->all_atoms);
}

In [10]:
my $i= 0;
open(my $fh, ">", "$pdbid\_dihe.txt");
printf $fh(" %7s %7s %7s \n" , qw(  phi   psi   omega));
my @dihe_deg = map{
                    $dihedrals[$_]->dihe_deg
} 0 .. $#dihedrals;

unshift @dihe_deg, 0;


while ($i+3 < $#dihe_deg+4){
    printf $fh ("%5i %7.2f %7.2f %7.2f \n", $i, map{$dihe_deg[$_]}($i, $i+1, $i+2));
  #if($dihe_deg[$i] > 110 and $dihe_deg[$i+1] > 150){
  #  printf ("shit %5i %7.2f %7.2f %7.2f \n", $i, map{$dihe_deg[$_]}($i, $i+1, $i+2));
  #      $dihedrals[$i]->print_pdb;
  #      $dihedrals[$i+1]->print_pdb;
  #}
    $i += 3;
} 
close($fh);

1


Warning: Use of uninitialized value in printf at reply input line 12.
